# Callable cells

Cleaner version of the magics in [magic_callbacks.ipynb](magic_callbacks.ipynb) in this directory.

In [ ]:
from IPython.core.magic import cell_magic, Magics, magics_class

@magics_class
class CallableCells(Magics):
    registry = {}
    
    @cell_magic
    def callable(self, line, cell):
        CallableCells.registry[line] = (cell, self.shell)
        self.shell.ex(cell)
        
    @staticmethod
    def run_cells(*names):
        for name in names:
            cell, shell = CallableCells.registry[name]
            shell.ex(cell)
            
    @staticmethod
    def callback(output, *names):
        def the_callback(w):
            with output:
                CallableCells.run_cells(*names)
        return the_callback

# Register
from IPython import get_ipython
ip = get_ipython()
ip.register_magics(CallableCells)

In [ ]:
x = 0

In [ ]:
%%callable inc

x += 1
print(f"inc x={x}")

In [ ]:
%%callable double

x *= 2
print(f"double x={x}")

In [ ]:
CallableCells.run_cells('double', 'inc', 'inc')
CallableCells.run_cells('inc')

In [ ]:
import ipywidgets as widgets
text = widgets.IntText()
output = widgets.Output()
button_reset = widgets.Button(description='Set x to...')
button_reset.on_click(CallableCells.callback(output, 'setx'))
button_double = widgets.Button(description='Double x')
button_double.on_click(CallableCells.callback(output, 'double'))
button_inc = widgets.Button(description='Inc x')
button_inc.on_click(CallableCells.callback(output, 'inc'))
widgets.VBox([widgets.HBox([button_reset, text]), widgets.HBox([button_double, button_inc]), output])

In [ ]:
%%callable setx
x = text.value
print(f"x set to {x}")